In [2]:
import networkx as nx
import sys
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per eseguire Girvan-Newman (Newman-Girvan) sul grafo
def girvan_newman(graph):
    """
    Rileva le comunità in un grafo utilizzando l'algoritmo Girvan-Newman.
    
    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
    
    Returns:
        list: Una lista di comunità, ciascuna rappresentata da un insieme di nodi.
    """
    # Copia il grafo per non modificare il grafo originale
    graph_copy = graph.copy()
    communities = []
    
    # Continua a dividere finché non ci sono più di un componente connesso
    while len(list(nx.connected_components(graph_copy))) == 1:
        # Calcola la betweenness centrality per ogni arco
        edge_betweenness = nx.edge_betweenness_centrality(graph_copy)
        
        # Trova l'arco con la massima betweenness
        edge_to_remove = max(edge_betweenness, key=edge_betweenness.get)
        
        # Rimuove l'arco con la massima betweenness
        graph_copy.remove_edge(*edge_to_remove)
    
    # Ottieni i componenti connessi (comunità) nel grafo dopo aver rimosso gli archi
    for component in nx.connected_components(graph_copy):
        communities.append(component)
    
    return communities

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Converti il grafo in un grafo non orientato (se è diretto)
graph = graph.to_undirected()

# Esegui Girvan-Newman per trovare le comunità
communities = girvan_newman(graph)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna le comunità come attributo dei nodi
for i, community in enumerate(communities):
    for node in community:
        gr.nodes[node]['community'] = str(i)  # Assegna l'ID della comunità

# Funzione per sanitizzare attributi e chiavi
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e gli attributi del grafo per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")  # Rimpiazza spazi e trattini
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)
    
    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Applica la sanitizzazione al grafo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_girvan_newman_communities.gml"
nx.write_gml(gr, output_path)



In [3]:
import sys
import networkx as nx
from networkx.algorithms.community import modularity
from sklearn.metrics import silhouette_score

# Imposta il percorso del progetto e importa la classe GraphConstructor
project_path = r"C:\Users\carcu\Desktop\Progetto Social\progetto_asnm-main\progetto_asnm-main"
sys.path.append(project_path)

from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per eseguire Girvan-Newman (Newman-Girvan) sul grafo
def girvan_newman(graph, max_communities=5, centrality_threshold=None):
    """
    Rileva le comunità in un grafo utilizzando l'algoritmo Girvan-Newman.
    
    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
        max_communities (int): Numero massimo di comunità da trovare.
        centrality_threshold (float): Soglia opzionale per la betweenness centrality.

    Returns:
        list: Una lista di comunità, ciascuna rappresentata da un insieme di nodi.
    """
    # Copia il grafo per non modificare il grafo originale
    graph_copy = graph.copy()
    while len(list(nx.connected_components(graph_copy))) < max_communities:
        # Calcola la betweenness centrality per ogni arco
        edge_betweenness = nx.edge_betweenness_centrality(graph_copy)
        
        # Filtra gli archi con centralità sopra la soglia (se specificata)
        if centrality_threshold:
            edges_to_remove = [
                edge for edge, centrality in edge_betweenness.items() if centrality > centrality_threshold
            ]
        else:
            # Rimuovi solo l'arco con la massima centralità
            edge_to_remove = max(edge_betweenness, key=edge_betweenness.get)
            edges_to_remove = [edge_to_remove]
        
        if not edges_to_remove:
            break  # Se non ci sono più archi da rimuovere, fermati

        # Rimuovi gli archi
        graph_copy.remove_edges_from(edges_to_remove)
    
    # Ottieni le comunità come componenti connesse
    communities = [set(component) for component in nx.connected_components(graph_copy)]
    return communities

# Funzione per sanitizzare attributi e chiavi
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e gli attributi del grafo per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")  # Rimpiazza spazi e trattini
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)
    
    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Converti il grafo in un grafo non orientato (se è diretto)
graph = graph.to_undirected()

# Esegui Girvan-Newman per trovare le comunità
max_communities = 5
centrality_threshold = 0.1
communities = girvan_newman(graph, max_communities=max_communities, centrality_threshold=centrality_threshold)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna le comunità come attributo dei nodi
for i, community in enumerate(communities):
    for node in community:
        gr.nodes[node]['community'] = str(i)  # Assegna l'ID della comunità

# Applica la sanitizzazione al grafo
sanitize_graph_attributes(gr)

# Calcola la modularità
modularity_score = modularity(graph, communities)
print(f"Modularità calcolata: {modularity_score}")

# Calcola l'indice di silhouette
labels = [gr.nodes[node]['community'] for node in gr.nodes]
adj_matrix = nx.to_numpy_array(gr)
silhouette_score_value = silhouette_score(adj_matrix, labels, metric='precomputed')
print(f"Indice di Silhouette calcolato: {silhouette_score_value}")

# Esporta il grafo in formato GML
output_path = "graph_with_girvan_newman_communities.gml"
nx.write_gml(gr, output_path)

print(f"Grafo salvato in: {output_path}")


Modularità calcolata: 0.49997442760023203
Indice di Silhouette calcolato: -0.731796317797313
Grafo salvato in: graph_with_girvan_newman_communities.gml
